# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [47]:
import pandas as pd
import pymongo

In [48]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [49]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [50]:
# Charger les catégories depuis le fichier JSON
import json

with open("./data/US_category_id.json", "r") as f:
    categories_data = json.load(f)

# Créer un dictionnaire {category_id: category_name}
categories_dict = {}
for item in categories_data["items"]:
    cat_id = int(item["id"])
    cat_name = item["snippet"]["title"]
    categories_dict[cat_id] = cat_name

print(f"Nombre de catégories chargées : {len(categories_dict)}")
print("Exemples de catégories :", list(categories_dict.items())[:3])

# Ajouter le nom de la catégorie au dataframe YouTube
df_youtube["category_name"] = df_youtube["category_id"].map(categories_dict)

# Supprimer les lignes avec des éléments vides
print(f"\nNombre de lignes avant nettoyage : {len(df_youtube)}")
df_youtube = df_youtube.dropna()
print(f"Nombre de lignes après suppression des valeurs manquantes : {len(df_youtube)}")

# Supprimer les doublons basés sur video_id
df_youtube = df_youtube.drop_duplicates(subset=['video_id'])
print(f"Nombre de lignes après suppression des doublons (video_id) : {len(df_youtube)}")

# Vérifier les données
print("\nAperçu des données avec catégories :")
print(df_youtube[["title", "category_id", "category_name"]].head())

Nombre de catégories chargées : 32
Exemples de catégories : [(1, 'Film & Animation'), (2, 'Autos & Vehicles'), (10, 'Music')]

Nombre de lignes avant nettoyage : 7992
Nombre de lignes après suppression des valeurs manquantes : 7992
Nombre de lignes après suppression des doublons (video_id) : 2364

Aperçu des données avec catégories :
                                               title  category_id  \
0  1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...           24   
1            iPhone X — Introducing iPhone X — Apple           28   
2                                        My Response           22   
3                          Apple iPhone X first look           28   
4                                  iPhone X (parody)           23   

          category_name  
0         Entertainment  
1  Science & Technology  
2        People & Blogs  
3  Science & Technology  
4                Comedy  


### Importer les données

In [51]:
# Supprimer la collection si elle existe déjà
collection.drop()

# Convertir le dataframe en dictionnaire et insérer dans MongoDB
data_to_insert = df_youtube[df_youtube["channel_title"]=="Apple"].to_dict('records')
result = collection.insert_many(data_to_insert)

print(f"✅ {len(result.inserted_ids)} documents insérés dans MongoDB")

# Vérifier l'insertion
print(f"\nNombre total de documents dans la collection : {collection.count_documents({})}")
print("\nExemple de document :")
print(collection.find_one())

✅ 3 documents insérés dans MongoDB

Nombre total de documents dans la collection : 3

Exemple de document :
{'_id': ObjectId('692ef08741e78d5dd66957f5'), 'video_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 13.09, 'category_name': 'Science & Technology'}


## Question 1  

In [52]:
# Question 1) Récupérer toutes les vidéos de la chaîne Apple.
videos_apple = list(collection.find({"channel_title": "Apple"}))

print(f"Nombre de vidéos de la chaîne Apple : {len(videos_apple)}")
print("\nExemples de vidéos :")
for video in videos_apple[:3]:
    print(f"- {video['title']} (vues: {video['views']})")

Nombre de vidéos de la chaîne Apple : 3

Exemples de vidéos :
- iPhone X — Introducing iPhone X — Apple (vues: 7860119)
- Apple Watch Series 3 + Apple Music — Roll — Apple (vues: 782543)
- iPhone 8 Plus — Portraits of Her  — Apple (vues: 117564)


## Question 2

In [53]:
# Question 2) Compter le nombre de catégories différentes
categories_distinctes = collection.distinct("category_name")

print(f"Nombre de catégories différentes : {len(categories_distinctes)}")
print("\nListe des catégories :")
for cat in sorted(categories_distinctes):
    print(f"- {cat}")

Nombre de catégories différentes : 1

Liste des catégories :
- Science & Technology


## Question 3

In [54]:
# Question 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

# Vérifier d'abord le format actuel des tags
exemple = collection.find_one()
print(f"Format actuel des tags : {type(exemple['tags'])}")
print(f"Exemple : {exemple['tags'][:100]}...")

# Mettre à jour tous les documents pour transformer les tags en listes
# Les tags sont séparés par des pipes (|)
for doc in collection.find():
    if isinstance(doc['tags'], str):
        tags_list = [tag.strip() for tag in doc['tags'].split('|') if tag.strip()]
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"tags": tags_list}}
        )

print("\n✅ Mise à jour terminée")

# Vérifier la transformation
exemple_apres = collection.find_one()
print(f"\nFormat après transformation : {type(exemple_apres['tags'])}")
print(f"Exemple : {exemple_apres['tags'][:5]}")

Format actuel des tags : <class 'str'>
Exemple : Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ...

✅ Mise à jour terminée

Format après transformation : <class 'list'>
Exemple : ['Apple', 'iPhone 10', 'iPhone Ten', 'iPhone', 'Portrait Lighting']


## Question 4

In [55]:
# Question 4) Récupérer les vidéos les plus vues.
videos_plus_vues = list(collection.find().sort("views", -1).limit(10))

print("Top 10 des vidéos les plus vues :\n")
for i, video in enumerate(videos_plus_vues, 1):
    print(f"{i}. {video['title']}")
    print(f"   Chaîne: {video['channel_title']} | Vues: {video['views']:,}")
    print()

Top 10 des vidéos les plus vues :

1. iPhone X — Introducing iPhone X — Apple
   Chaîne: Apple | Vues: 7,860,119

2. Apple Watch Series 3 + Apple Music — Roll — Apple
   Chaîne: Apple | Vues: 782,543

3. iPhone 8 Plus — Portraits of Her  — Apple
   Chaîne: Apple | Vues: 117,564



## Question 5

In [56]:
# Question 5) Compter le nombre moyen de vues en fonction de la catégorie.
pipeline = [
    {
        "$group": {
            "_id": "$category_name",
            "moyenne_vues": {"$avg": "$views"},
            "nombre_videos": {"$sum": 1}
        }
    },
    {
        "$sort": {"moyenne_vues": -1}
    }
]

resultats = list(collection.aggregate(pipeline))

print("Nombre moyen de vues par catégorie :\n")
for cat in resultats:
    print(f"{cat['_id']:25} | Moyenne: {cat['moyenne_vues']:>12,.0f} vues | Vidéos: {cat['nombre_videos']}")

Nombre moyen de vues par catégorie :

Science & Technology      | Moyenne:    2,920,075 vues | Vidéos: 3


## Question 6 

In [57]:
# Question 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.
pipeline = [
    {
        "$group": {
            "_id": "$channel_title",
            "moyenne_likes": {"$avg": "$likes"},
            "nombre_videos": {"$sum": 1}
        }
    },
    {
        "$sort": {"moyenne_likes": -1}
    },
    {
        "$limit": 10
    }
]

top_chaines = list(collection.aggregate(pipeline))

print("Top 10 des chaînes avec la plus grande moyenne de likes :\n")
for i, chaine in enumerate(top_chaines, 1):
    print(f"{i}. {chaine['_id']}")
    print(f"   Moyenne de likes: {chaine['moyenne_likes']:>10,.0f} | Nombre de vidéos: {chaine['nombre_videos']}")
    print()

Top 10 des chaînes avec la plus grande moyenne de likes :

1. Apple
   Moyenne de likes:     67,247 | Nombre de vidéos: 3

